# Smart Optimization to Save Computing Time

In [ ]:
???

<table style="width:100%; height:90%">
      <tr>
    <th>Grid Search</th>
    <th>Smart Search</th>
  </tr>
  <tr>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
    <td><img src="src/08_Table_Smart Optimization.png" alt="Parametrize the Strategy" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

In [2]:
with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [3]:
model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [4]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Adj Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,,
2016-12-08,61.299999,61.580002,60.840000,61.009998,55.705235,21220800,1.549141,UP
2016-12-09,61.180000,61.990002,61.130001,61.970001,56.581772,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,56.764374,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,57.503944,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,57.230022,30352700,-0.159793,DOWN
...,...,...,...,...,...,...,...,...
2023-12-18,369.450012,373.000000,368.679993,372.649994,372.649994,21802900,0.163429,UP
2023-12-19,371.489990,373.260010,369.839996,373.260010,373.260010,20603700,-0.712324,DOWN
2023-12-20,375.000000,376.029999,370.529999,370.619995,370.619995,26316700,0.781714,UP


## Parametrize & optimize the investment strategy

### Create Strategy class

In [5]:
from backtesting import Strategy

In [6]:
class SimpleRegression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create `Backtest` class

In [7]:
from backtesting import Backtest

In [8]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [9]:
bt = Backtest(df_explanatory, SimpleRegression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [10]:
import multiprocessing as mp
mp.set_start_method('fork')

In [ ]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy=[0, 10],
    limit_sell=[-10, 0],
    maximize='Return [%]',
    method='skopt',
    max_tries=500,
    random_state=0,
    return_heatmap=True,
    return_optimization=True
    )

### Which are the best values for the parameters?

In [ ]:
dff = heatmap.reset_index()
dff = dff.pivot(index='limit_buy', columns='limit_sell', values='Return [%]')
dff

In [ ]:
dff.style.highlight_null(props='background-color: transparent; color: transparent')

In [12]:
import numpy as np

In [ ]:
dff.sort_index(axis=1, ascending=False)\
  .style.format(precision=0)\
  .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))\
  .highlight_null(props='background-color: transparent; color: transparent')

### How to interpret smart optimization?

#### Number of evaluations

https://kernc.github.io/backtesting.py/doc/examples/Parameter%20Heatmap%20&%20Optimization.html

In [ ]:
from skopt.plots import plot_evaluations

_ = plot_evaluations(optimize_result, bins=10)

#### Density

In [ ]:
from skopt.plots import plot_objective

_ = plot_objective(optimize_result, n_points=10)

## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="08C_Smart Optimization to Save Computing Time.ipynb">notebook</a>.